In [2]:
#import the necessary functions
import os
import subprocess as subp #for handling shell commands
from subprocess import Popen, PIPE  # replacement for os.system()
import numpy as np

def _popen(cmd, data_in, data_out=PIPE):
    """
    Helper function subprocess.popen replaces os.system 
    - gives better input/output process control
    usage: pass <data_in> to process <cmd> and return results
    from rgbeimage.py (Thomas Bleicher 2010)
    """
    cmd = str(cmd) #removes unicode
    p_o = Popen(cmd, bufsize=-1, stdin=PIPE, stdout=data_out, stderr=PIPE)
    data, err = p_o.communicate(data_in)
    if err:
        return 'message: '+err.strip()
    if data:
        return data

In [3]:
#set working directory path
path = r'C:\Radiance\bin\pv_plant'
os.chdir(path)
cwd = os.getcwd()
print "Current working directory: " + cwd

Current working directory: C:\Radiance\bin\pv_plant


In [7]:
azi = -45
tilt = 0
l = 1.6 #length of a panel
h = 1.0 #height of a panel
mh = 1.0 #mount height

#calculate the centre point of each panel
u = (l*np.sin(np.deg2rad(90-azi)))/2 #x coordinate
v = (l*np.cos(np.deg2rad(90-azi)))/-2 #y coordinate
w = mh + (h/2) #z coordinate

#calculate the normal vector
norm_x = np.sin(np.deg2rad(90-azi))
norm_y = np.cos(np.deg2rad(90-azi))
norm_z = np.cos(np.deg2rad(90-tilt))


#write the calculated normal vector values to the file
file = open("normal_vec.dat","w")
file.write('%s ' %norm_x + '%s ' %norm_y + '%s' %norm_z)
file.close() 

#write the measurement points to a file
file = open("measure_front.dat","w")
for j in range(0,10,2):
    meas_x = j + u
    
    for i in range(0,10,2):
        meas_y = i + v
        
        #x y z x' y' z'
        file.write('%s ' %meas_x + \
                   '%s ' %meas_y + \
                   '%s ' %w + \
                   '%s ' %norm_x + \
                   '%s ' %norm_y + \
                   '%s' %norm_z + \
                   '\n')

file.close()

#run the bash script
comd = "bash pv_script.bash"
error = _popen(comd,None)

if error is not None:
    print error

In [10]:
comd = "rad -o qt pv_control.rif"
error = _popen(comd,None)

if error is not None:
    print error

comd = "rtrace -h -w -ab 2 -I -ad 512 -as 128 -ov pv_control.oct < measure_front.dat \
        | rcalc -f irr_formula.dat > irr_front.dat"
error = _popen(comd,None)

if error is not None:
    print error

set ROPT=-dp 1024 -ar 1706 -ms 0.028 -ds .3 -dt .1 -dc .5 -dr 1 -ss 1 -st .1 -af illum.amb -aa .1 -ad 1536 -as 392 -av 17 17 17 -lr 8 -lw 1e-4
	rvu -vf bev.vf $ROPT -ps 4 -pt .08 -R pv_control.rif -o qt -pe 0.0288702 pv_control.oct

message: rtrace: fatal - command line error at 'pv_control.oct'
